# 데이터 추가, 삭제, 갱신, 데이터 정합성

## INSERT: 행 추가하기

* 테이블에 데이터를 추가하고자 할 때 INSERT구문을 사용하면 된다.

In [ ]:
# 예시 구문
INSERT INTO table_name (column1, column2, column3, ...) VALUES
(value1, value2, value3);

In [ ]:
# 여러 줄을 추가하고자 할 때
INSERT INTO table_name (column1, column2, column3, ...) VALUES
(value1, value2, value3, ...),
(value4, value5, value6, ...);

## DELETE: 행 삭제하기

* 데이터의 행을 삭제하고자 할 때 DELETE구문을 사용하면 된다.

In [ ]:
# 예시 구문
DELETE FROM table_name
WHERE some_column = some_value;

* 데이터를 삭제하는 데 WHERE절이 사용되는 이유는 데이터를 삭제할 때 어떤 데이터를 삭제하라는 조건이 필요할 경우가 존재하기 때문이다.

In [ ]:
# 특정 조건에 대한 데이터 행 삭제
DELETE FROM table_name
WHERE 상품 번호 = 'a003';

In [ ]:
# 성별이 'f'인 데이터를 모두 삭제하게 될 경우
DELETE FROM PRODUCT
WHERE 성별 = 'f';

* 위 쿼리를 실행하게 되면 성별이 'f'인 데이터가 모두 삭제되므로 데이터를 삭제하는 경우 신중히 작업해야 한다.

## UPDATE: 데이터 갱신하기

* 데이터의 값을 변경하고자 할 때 UPDATE구문을 사용한다.

In [ ]:
# 예시 구문
UPDATE table_name
SET column_name = 'new_value'
WHERE condition;

* 데이터를 업데이트 할 때도 특정 조건을 만족하는 데이터를 갱신하는 경우가 많다.

In [ ]:
# 특정 조건의 데이터를 업데이트
UPDATE product
SET 원가 = 70000
WHERE 상품 번호 = 'a002';

UPDATE product
SET 원가 = 70000
카테고리 = '피트니스'
WHERE 상품 번호 = 'a002';

## PROCEDURE

* 매크로처럼 반복되는 내용을 하나의 단위로 생성할 수 있는데, 이를 PROCEDURE라고 한다.

In [ ]:
# 취소가 'Y'인 데이터의 매출액에 -1을 곱하면 매출액을 음수로 변경
UPDATE product
SET 원가 = (-1)*원가
WHERE 취소 여부 = 'Y';

* 위 쿼리를 오늘 실행하고 내일도 실행한다면 다음의 절차로 이뤄질 것이다.
* 예를 들어, 2023.01.01에 실행하게 되면 2023.01.01까지의 매출 건 중 취소 여부가 'Y'인 데이터의 원가가 음수(-)처리된다.
* 위 요청은 매일 처리되는 일 배치이므로 2023.01.02 매출 건에도 실행되어야 한다.
* 2023.01.02에 해당 쿼리를 사용하면 다음과 같이 작성할 수 있다.

In [ ]:
# 2023.01.02에 위 쿼리를 활용
UPDATE product
SET 원가 = (-1)*원가
WHERE 취소 여부 = 'Y';
AND 판매 일자 = CURDATE() - 1;

* 2023.01.01의 취소 건의 원가에 다시 -1이 곱해지므로 2023.01.01 취소 건의 원가는 +가 된다.
* 이런 과정이 반복될 것이므로 한 가지 조건이 더 추가되어야 한다.
* **CURDATE**: 현재 일자를 알려주는 MySQL함수

In [ ]:
# 프로시저 생성 예시 구문
DELIMITER $$
CREATE PROCEDURE 프로시저 명()
BEGIN
쿼리;
END $$
DELIMITER ;

* 위에 반복했던 쿼리를 sales_minus라는 프로시저이름으로 생성해보면 아래와 같다.

In [ ]:
# sales_minus라는 이름의 프로시저 생성
DELIMITER $$
CREATE PROCEDURE sales_minus()
BEGIN
UPDATE product
SET 원가 = (-1)*원가
WHERE 취소 여부 = 'Y';
AND 판매 일자 = CURDATE() - 1
END $$
DELIMITER ;

## VIEW

* VIEW는 테이블을 직접 생성하지 않고 SELECT 문의 출력 결과를 보여준다.

In [ ]:
# 기존 쿼리문
SELECT 주문 번호
FROM DB.SALES
WHERE 취소 여부 = 'Y';

* 위 쿼리를 실행하면 SALES 테이블에서 최소여부가 'Y'인 데이터만 조회될 것이다.
* 하지만 보안상의 이유로 우리는 협력사에 SALES 테이블을 조회할 수 있는 권한을 줄 수 없다.
* 이때 VIEW로 문제를 해결할 수 있다.

In [ ]:
# 뷰 사용 예시 구문
CREATE VIEW DB.view_name
AS
SELECT-STATEMENT;

In [ ]:
# cancel_prodno라는 뷰 생성
CREATE VIEW DB.cancel_prodno
AS
SELECT 주문 번호
FROM DB.SALES
WHERE 취소 여부 = 'Y';

* 생성된 View는 테이블과 동일하게 사용 가능하다.

In [ ]:
# 생성된 뷰는 테이블과 동일하게 사용 가능
SELECT *
FROM DB.cancel_prodno;

### VIEW의 특징

* 가상의 테이블, 사용자의 입장에서는 테이블과 동일하게 보이지만, 뷰는 실제 데이터를 가지고 있지 않음
* 실제 테이블에 링크되는 개념임
* 엑세스 제한을 위해서 주로 사용(테이블 자체적으로는 행, 열 단위로 권한 부여할 수 없음)

## 데이터 정합성

* 데이터 정합성: 데이터의 값이 일치함을 의미
* 예를 들어, 매출 테이블에서 연도별 매출을 계산한다는 것 -> 전체 매출을 연도별로 나눈다는 것 의미
* 반대로 연도별로 나눈 매출의 합은 전체 매출과 같아야 한다.

## MICE란?

* 각 항목들이 상호 배타적이면서 모였을 때는 완전하게 합쳐지는 것
* 대부분의 분석은 전체에서 부분으로 나누어 살펴보게 된다. 이때 부분의 합은 전체와 일치해야 한다.
* 분석하다 보면 부분의 합이 전체의 합보다 크거나 작은 경우가 빈번히 발생한다.
* 데이터 신뢰성을 확보하여면 부분의 합이 전체와 일치하는지 항상 확인하면서 분석을 진행해야 한다.
* 매번 데이터를 출력할 때마다 정합성을 확인한다는 것이 번거로우나, 장기적으로 손해보는 일이 아니므로 항상 정합성을 확인하는 습관을 기르는 것이 좋다.